# Imports

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from scikeras.wrappers import KerasClassifier

import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


2025-08-30 15:31:09.367512: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-30 15:31:09.375461: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-30 15:31:09.451220: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-30 15:31:09.501808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756578669.556740   60464 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756578669.57

# Constants

In [2]:
SAMPLE_SIZE = 50000
VOCAB_SIZE = 5000  # Limita o vocabulário às 30k palavras mais comuns
MAX_LEN = 50       # Comprimento máximo da sequência após padding
EMBEDDING_DIM = 64

# Utils

In [3]:
def create_model(vocab_size, lstm_units=64, embedding_dim=128, dropout_rate=0.3):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_shape=(MAX_LEN,)),  # Adicione input_length
        LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def clean_text(text):
    text = text.lower()  # Converte para minúsculas
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove tags HTML
    text = re.sub(r'@\w+', '', text)  # Remove menções
    text = re.sub(r'#', '', text)  # Remove o símbolo de hashtag
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove caracteres especiais e números
    return text

# Mapear rótulos numéricos para texto
def map_label(label):
    return "Positivo" if label == 1 else "Negativo"

# Load Data

In [4]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv',
                     encoding='latin-1',
                     header=None)

df = df.sample(n=SAMPLE_SIZE, random_state=42)

In [5]:
len(df)

50000

In [6]:
df.head()

,0,1,2,3,4,5
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


In [7]:
len(df)

50000

In [8]:
df.head()

,0,1,2,3,4,5
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


# Preprocess Data

## Adjust Column Name

In [9]:
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

In [10]:
df.head()

,target,ids,date,flag,user,text
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


## Adjust Target

In [11]:
df['target'].value_counts()

target
4    25014
0    24986
Name: count, dtype: int64

In [12]:
df['target'] = df['target'].apply(lambda x: 1 if x == 4 else 0)

In [13]:
df['target'].value_counts()

target
1    25014
0    24986
Name: count, dtype: int64

## Clean Text

In [14]:
df['text'].iloc[0]

'@chrishasboobs AHHH I HOPE YOUR OK!!! '

In [15]:
df['cleaned_text'] = df['text'].apply(clean_text)

In [16]:
df['cleaned_text'].iloc[0]

' ahhh i hope your ok '

## Split Train, Val and Test Data

In [17]:
X = df['cleaned_text']
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Grid Search

In [18]:
all_results = []

In [21]:
vocab_sizes_to_test = [5000, 10000, 20000, 30000] # Tamanhos do vocabulário
param_grid = {
    'batch_size': [32, 64, 128, 256],
    'model__embedding_dim': [32, 64, 128],
    'model__lstm_units': [1, 2, 4, 8, 16, 32, 64, 128],
    'model__dropout_rate': [0.2, 0.4]
}

# param_grid = {
#     'batch_size': [32],
#     'model__embedding_dim': [32],
#     'model__lstm_units': [1],
#     'model__dropout_rate': [0.2]
# }

In [22]:
for vocab_size in vocab_sizes_to_test:
    print("-" * 50)
    print(f"INICIANDO BUSCA PARA VOCAB_SIZE = {vocab_size}")
    print("-" * 50)

    # 3.1. Pré-processamento específico para o vocab_size atual
    tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
    tokenizer.fit_on_texts(X_train)
    X_train_pad = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=MAX_LEN, padding='pre')

    # 3.2. Configuração do Grid Search para a iteração atual
    # Passamos o vocab_size como um parâmetro fixo para a função create_model
    model_cv = KerasClassifier(
        model=create_model,
        verbose=0,
        vocab_size=vocab_size  # Parâmetro fixo para esta busca
    )
    
    grid = GridSearchCV(estimator=model_cv, param_grid=param_grid, cv=3, verbose=2)
    
    grid_result = grid.fit(X_train_pad, y_train)

    # 3.3. Armazenar os resultados
    result = {
        'vocab_size': vocab_size,
        'best_score': grid_result.best_score_,
        'best_params': grid_result.best_params_
    }
    all_results.append(result)

[CV] END batch_size=128, model__dropout_rate=0.2, model__embedding_dim=32, model__lstm_units=4; total time=   6.8s


/home/victor/anaconda3/envs/dl/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[CV] END batch_size=128, model__dropout_rate=0.2, model__embedding_dim=32, model__lstm_units=4; total time=   6.3s


/home/victor/anaconda3/envs/dl/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[CV] END batch_size=128, model__dropout_rate=0.2, model__embedding_dim=32, model__lstm_units=8; total time=   9.0s


...

/home/victor/anaconda3/envs/dl/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[CV] END batch_size=256, model__dropout_rate=0.4, model__embedding_dim=128, model__lstm_units=128; total time=  11.3s


/home/victor/anaconda3/envs/dl/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[CV] END batch_size=256, model__dropout_rate=0.4, model__embedding_dim=128, model__lstm_units=128; total time=  13.4s


/home/victor/anaconda3/envs/dl/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:

print("\n" + "=" * 50)
print("RESULTADO FINAL DE TODOS OS GRID SEARCHES")
print("=" * 50)

# Encontrar o melhor resultado geral
best_overall_result = max(all_results, key=lambda x: x['best_score'])

print(f"Melhor Acurácia Global: {best_overall_result['best_score']:.4f}\n")
print("Melhor combinação de hiperparâmetros encontrada:")
print(f"  - Tamanho do Vocabulário (vocab_size): {best_overall_result['vocab_size']}")
# Os parâmetros do grid estão aninhados, então precisamos extraí-los
best_grid_params = best_overall_result['best_params']
for key, value in best_grid_params.items():
    # Remover o prefixo 'model__' para uma exibição mais limpa
    clean_key = key.replace('model__', '')
    print(f"  - {clean_key}: {value}")


RESULTADO FINAL DE TODOS OS GRID SEARCHES
Melhor Acurácia Global: 0.7728

Melhor combinação de hiperparâmetros encontrada:
  - Tamanho do Vocabulário (vocab_size): 10000
  - batch_size: 32
  - dropout_rate: 0.2
  - embedding_dim: 128
  - lstm_units: 32


# ----------------------------------------------------------------------------------------------------------------------